# **Решение уравнение Якоба-Гамльтона-Белламана**

In [ ]:
def solve_hjb(t_grid, x_grid, mu_portfolio, sigma_portfolio, r, gamma):

    Nt = len(t_grid)
    Nx = len(x_grid)
    V = np.zeros((Nt, Nx))

    V[:, 0] = np.log(x_grid[0] + 1e-10)
    V[-1, :] = np.log(x_grid + 1e-10)

    dt = t_grid[1] - t_grid[0]
    dx = x_grid[1] - x_grid[0]

    V_min = np.log(np.min(x_grid) + 1e-10)
    V_max = np.log(np.max(x_grid) + 1e-10) * 20

    for i in reversed(range(Nt - 1)):
        mu = mu_portfolio[i]
        sigma = sigma_portfolio[i]
        for j in range(1, Nx - 1):
            dVdx = (V[i+1, j+1] - V[i+1, j-1]) / (2*dx)
            d2Vdx2 = (V[i+1, j+1] - 2*V[i+1, j] + V[i+1, j-1]) / (dx**2)
            term1 = mu * x_grid[j] * dVdx
            term2 = 0.5 * (sigma ** 2) * (x_grid[j] ** 2) * d2Vdx2
            term3 = r * V[i+1, j]
            V[i, j] = V[i+1, j] + dt * (term1 + term2 - term3)
            if not np.isfinite(V[i, j]):
                V[i, j] = V_min
            V[i, j] = np.clip(V[i, j], V_min, V_max)
        V[i, 0] = V_min
        V[i, -1] = V_max
    return V

def plot_value_function(t_grid, x_grid, V):

    plt.figure(figsize=(10, 6))
    T, X = np.meshgrid(t_grid, x_grid, indexing='ij')
    cp = plt.contourf(T, X, V, cmap='viridis')
    plt.colorbar(cp)
    plt.xlabel('Время')
    plt.ylabel('Капитал')
    plt.title('Значения функции стоимости V(t,x) из HJB (стабилизировано)')
    plt.show()

t_grid = np.linspace(0, 1, 120)
x_grid = np.linspace(1e5, 1.5e6, 400)
mu_portfolio = np.full(len(t_grid), 0.07)
sigma_portfolio = np.full(len(t_grid), 0.2)
r = 0.03
gamma = 3.0

V = solve_hjb(t_grid, x_grid, mu_portfolio, sigma_portfolio, r, gamma)
plot_value_function(t_grid, x_grid, V)